In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import pygame
import threading
import mediapipe as mp

# Load trained model
model = tf.keras.models.load_model("drowsiness_detector_model.keras")

# Initialize Pygame for alarm
pygame.mixer.init()
pygame.mixer.music.load("alarm.mp3")

# Constants
img_size = (64, 64)
threshold_frames = 10  # trigger alarm if closed for this many frames

# MediaPipe setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Eye landmarks from MediaPipe
LEFT_EYE = [362, 385, 387, 263, 373, 380]
RIGHT_EYE = [33, 160, 158, 133, 153, 144]

# Alarm function
def play_alarm():
    if not pygame.mixer.music.get_busy():
        pygame.mixer.music.play()

# Eye crop with looser padding
def crop_eye(frame, landmarks, indices):
    h, w, _ = frame.shape
    points = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in indices]
    x_coords, y_coords = zip(*points)

    eye_width = max(x_coords) - min(x_coords)
    eye_height = max(y_coords) - min(y_coords)

    # Looser padding
    padding_x = int(eye_width * 1.5)
    padding_y = int(eye_height * 1.5)

    x_center = int(np.mean(x_coords))
    y_center = int(np.mean(y_coords))

    x_min = max(x_center - padding_x, 0)
    x_max = min(x_center + padding_x, w)
    y_min = max(y_center - padding_y, 0)
    y_max = min(y_center + padding_y, h)

    return frame[y_min:y_max, x_min:x_max]

# Webcam init
cap = cv2.VideoCapture(0)
closed_eyes_frame_count = 0

print("[INFO] Press 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)
    eye_closed_detected = False

    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark

        for eye_name, eye_indices in zip(["Left", "Right"], [LEFT_EYE, RIGHT_EYE]):
            eye_crop = crop_eye(frame, landmarks, eye_indices)

            if eye_crop.size == 0:
                continue

            # Resize and grayscale
            eye_resized = cv2.resize(eye_crop, img_size)
            eye_gray = cv2.cvtColor(eye_resized, cv2.COLOR_BGR2GRAY)
            eye_gray = cv2.cvtColor(eye_gray, cv2.COLOR_GRAY2RGB)

            # Normalize and prepare input
            eye_input = eye_gray / 255.0
            eye_input = np.expand_dims(eye_input, axis=0)

            prediction = model.predict(eye_input, verbose=0)[0][0]
            label = "Closed" if prediction > 0.5 else "Open"  # threshold lowered
            color = (0, 255, 0) if label == "Open" else (0, 0, 255)

            if label == "Closed":
                eye_closed_detected = True

            # Draw label
            cv2.putText(frame, f"{eye_name}: {label} ({prediction:.2f})",
                        (10, 30 if eye_name == "Left" else 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            # Show eye crop (grayscale)
            cv2.imshow(f"{eye_name} Eye Crop", eye_gray)

    # Drowsiness logic
    if eye_closed_detected:
        closed_eyes_frame_count += 1
    else:
        closed_eyes_frame_count = 0

    if closed_eyes_frame_count >= threshold_frames:
        threading.Thread(target=play_alarm).start()
        closed_eyes_frame_count = 0

    # Show full frame
    cv2.imshow("Drowsiness Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()
